In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import threading
import bs4 as bs
import requests
import yfinance as yf
import datetime

## Group Assignment
### Team Number: 11
### Team Member Names: Bob Li, Lucas Zaharia, Charles Chen
### Team Strategy Chosen: RISKY

In [2]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]
df = pd.DataFrame (tickers, columns = ['0'])

ticker_list=df
print(ticker_list)

        0
0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
..    ...
498   YUM
499  ZBRA
500   ZBH
501  ZION
502   ZTS

[503 rows x 1 columns]


In [3]:
# Assign Start and End date for the analysis
start_date="2022-11-02"
end_date= "2022-12-30"

# Determine the minimum number of trading days required to be considered
apple =yf.Ticker("AAPL")
apple_hist = apple.history(start= start_date, end= end_date)
apple_hist.dropna(inplace=True)
min_data_len = len(apple_hist.index)

In [4]:
# Extract first trading day
apple_hist.reset_index(inplace=True)
first_trading_day = str(apple_hist["Date"][0])
first_trading_day = first_trading_day[0:10]

## Note on above code block

### You may see that there are outputs saying that certain tickers may be delisted. This is expected and normal. Yahoo Finance does this automatically. You may also notice that there are multiple delisted messages for the same ticker. Please note that this is not because we are retrieving the same data multiple times from the same ticker. This is because we use threading to split up calculating monthly volume, so when we call Yahoo Finance on each month, we get the same delisted message.

In [5]:
##New Code

In [6]:
print(ticker_list)

        0
0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
..    ...
498   YUM
499  ZBRA
500   ZBH
501  ZION
502   ZTS

[503 rows x 1 columns]


In [7]:
ticker_list=ticker_list.drop_duplicates()
print(ticker_list)

        0
0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
..    ...
498   YUM
499  ZBRA
500   ZBH
501  ZION
502   ZTS

[503 rows x 1 columns]


In [8]:
# Remove empty entries
ticker_list=ticker_list.replace('N/A',np.NaN)
ticker_list=ticker_list.dropna()

# Reset index
ticker_list.reset_index(drop=True, inplace=True)

In [9]:
# Create empty column called score
ticker_list['score']=''

# Calculates and insert score for each ticker in the dataframe
# Consumes Ticker list dataframe, index, start and end date
def eval_ticker(ticker_list,i,start_date, end_date):
    ticker_name=ticker_list.iloc[i,0]
    ticker=yf.Ticker(ticker_name)
    company=ticker.history(start= start_date, end= end_date)
    company=company["Close"]
    company=company.pct_change()
    company=company.dropna()
    standard_deviation=company.std()
    score=standard_deviation
    ticker_list['score'][i]=score

In [10]:
# Create a list of threads
threads = []

# Start Threading in a loop
for i in range(0,len(ticker_list)):
    # Create a new thread
    t = threading.Thread(target=eval_ticker, args=[ticker_list,i,start_date, end_date])
    
    # Start thread
    t.start()
    
    # Add thread to thread list
    threads.append(t)

# Stop all threads in list
for thread in threads:
    thread.join()


- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [11]:
# Sort the dataframe by score in decreasing order
ticker_list.sort_values(by=['score'], ascending=False, inplace=True)

# Reset the index
ticker_list.reset_index(drop=True,inplace=True)
ticker_list.dropna(inplace=True)
print(ticker_list)

        0     score
0     LNC  0.062582
1    BBWI  0.053823
2     FIS  0.051827
3    ETSY   0.04805
4    MRNA  0.047798
..    ...       ...
496    CL  0.008897
497   PEP  0.008652
498   JNJ  0.008596
499   MCD  0.008375
500   KHC  0.008068

[501 rows x 2 columns]


In [12]:
# Generates a dictionary where the key is the stock ticker and the associated value
# is a DataFrame containing daily close prices
# Parameters:
#  - ticker_lst (List): List containing stock tickers
# Returns:
#  - df_dict (Dictionary): Dictionary containing stock prices
def generate_stock_df_dict(ticker_lst):
    # Variables
    hist_interval = "1d"  
    df_dict = {}
    
    # Loop through each ticker in ticker_lst
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        
        # Get stock prices
        stock_hist = stock.history(start=start_date, interval=hist_interval)
        
        # Drop empty rows
        stock_hist.dropna(inplace=True)
        
        # Add DF with just Close column to dict
        df_dict[ticker] = stock_hist[["Close"]]
        
    # Return dict
    return df_dict

# Finds correlation in expected returns between two stocks when given a dictionary
# containing their stock prices.
# Parameters:
#  - ticker (String): Ticker of first stock
#  - second_ticker (String): Ticker of second stock
#  - df_dict (Dictionary): Dictionary containing stock prices
# Returns:
#  - returns_corr (Float): Correlation between expected returns
def find_corr(ticker, second_ticker, df_dict):
    # Variables
    ls = "_" + ticker
    rs = "_" + second_ticker
    
    # Join the price DataFrames for the two stocks
    prices = df_dict[ticker].join(df_dict[second_ticker], lsuffix=ls, rsuffix=rs)
    
    # Drop empty rows
    prices.dropna(inplace=True)
    
    # Get daily returns and drop first row
    daily_returns = prices.pct_change()
    daily_returns = daily_returns.iloc[1:]
    
    # Extract correlation
    returns_corr = daily_returns.corr().iloc[0, 1]
    
    # Return correlation
    return returns_corr

# Generates a list of portfolios, where each portfolio is composed of the 11 most
# correlated stocks to each stock in ticker_lst as well as the stock in ticker_lst
# Parameters:
#  - ticker_lst (List): List of tickers with length of at least 12
#  - df_dict (Dictionary): Dictionary containing stock prices
# Returns:
#  - portfolios (2D List): List of portfolios
def generate_portfolios(ticker_lst, df_dict):
    # Create empty list
    portfolios = []
    
    # Loop through each ticker in ticker_lst
    for ticker in ticker_lst:
        # Copy ticker_lst to avoid reference problems
        ticker_lst_copy = ticker_lst.copy()
        
        # Remove current ticker from copy
        ticker_lst_copy.remove(ticker)
        
        # Create a new DF where the index is ticker_lst_copy
        corr_df = pd.DataFrame(index=ticker_lst_copy)
        
        # Name the index Stock
        corr_df.index.name = "Stock"
        
        # Create a new column called Corr where each value is initially -2
        corr_df["Corr"] = -2
        
        # Loop through each ticker in ticker_lst_copy
        for second_ticker in ticker_lst_copy:
            # Find correlation between tickers in outer and inner for loops
            corr = find_corr(ticker, second_ticker, df_dict)
            
            # Place correlation value in appropriate location in DF
            corr_df.loc[second_ticker, "Corr"] = corr
        
        # Reset index
        corr_df.reset_index(inplace=True)
        
        # Sort DF values by Corr column from most correlated to least correlated
        corr_df.sort_values(by="Corr", ascending=False, inplace=True)
        
        # Create a new one element list containing ticker in outer for loop
        current_portfolio = [ticker]
        
        # Extract 11 most correlated stocks as a list
        additional_stocks = list(corr_df.head(11).Stock)
        
        # Extend the one element list to have the correlated stocks
        current_portfolio.extend(additional_stocks)
        
        # Append current portfolio to list of portfolios
        portfolios.append(current_portfolio)
    
    # Return list of portfolios
    return portfolios



In [13]:
print(ticker_list)

        0     score
0     LNC  0.062582
1    BBWI  0.053823
2     FIS  0.051827
3    ETSY   0.04805
4    MRNA  0.047798
..    ...       ...
496    CL  0.008897
497   PEP  0.008652
498   JNJ  0.008596
499   MCD  0.008375
500   KHC  0.008068

[501 rows x 2 columns]


In [14]:
# Keep 25 most risky stocks
ticker_list = ticker_list.head(25)
print(ticker_list)
# Convert tickers into list
risky_tickers = list(ticker_list['0'])

# Generate dictionary of stock prices
stock_price_dict = generate_stock_df_dict(risky_tickers)

# Generate 25 risky portfolios
portfolios = generate_portfolios(risky_tickers, stock_price_dict)

       0     score
0    LNC  0.062582
1   BBWI  0.053823
2    FIS  0.051827
3   ETSY   0.04805
4   MRNA  0.047798
5   SEDG  0.046646
6   TSLA  0.045695
7    CCL  0.045079
8    WBD  0.043857
9   DISH  0.043726
10  MTCH  0.043711
11  CDAY  0.042376
12  PARA  0.041821
13  NVDA  0.041819
14   CZR  0.041487
15  SIVB  0.041484
16  CTLT  0.041407
17   VFC  0.041299
18  GNRC  0.040577
19    ON  0.038385
20   AMD  0.038013
21  LUMN  0.038008
22  EPAM  0.037739
23  ENPH  0.037601
24  TROW  0.037251


In [15]:

print(portfolios,risky_tickers, stock_price_dict,ticker_list)

[['LNC', 'FIS', 'LUMN', 'EPAM', 'GNRC', 'DISH', 'TROW', 'MTCH', 'CTLT', 'PARA', 'WBD', 'VFC'], ['BBWI', 'AMD', 'PARA', 'TROW', 'CZR', 'ON', 'VFC', 'CCL', 'SEDG', 'GNRC', 'WBD', 'NVDA'], ['FIS', 'LNC', 'LUMN', 'EPAM', 'MTCH', 'DISH', 'CTLT', 'TROW', 'GNRC', 'ON', 'PARA', 'SIVB'], ['ETSY', 'CDAY', 'AMD', 'NVDA', 'CCL', 'CZR', 'ON', 'TROW', 'GNRC', 'MTCH', 'TSLA', 'SIVB'], ['MRNA', 'NVDA', 'EPAM', 'SIVB', 'MTCH', 'CZR', 'AMD', 'CDAY', 'SEDG', 'ENPH', 'ON', 'ETSY'], ['SEDG', 'ENPH', 'TROW', 'NVDA', 'CZR', 'AMD', 'GNRC', 'ON', 'CDAY', 'ETSY', 'EPAM', 'CCL'], ['TSLA', 'NVDA', 'CDAY', 'CCL', 'SIVB', 'AMD', 'ETSY', 'GNRC', 'CZR', 'MTCH', 'EPAM', 'ON'], ['CCL', 'AMD', 'CZR', 'NVDA', 'ETSY', 'CDAY', 'ON', 'PARA', 'GNRC', 'TROW', 'VFC', 'SIVB'], ['WBD', 'PARA', 'DISH', 'VFC', 'TROW', 'CCL', 'GNRC', 'CZR', 'MTCH', 'CDAY', 'SIVB', 'TSLA'], ['DISH', 'TROW', 'CZR', 'PARA', 'EPAM', 'MTCH', 'VFC', 'ON', 'GNRC', 'WBD', 'SIVB', 'NVDA'], ['MTCH', 'EPAM', 'SIVB', 'CZR', 'TROW', 'NVDA', 'ON', 'GNRC', 'AMD',

In [16]:
# Stores portfolio standard deviations
portfolio_stds = []

# Builds a portfolio with $500000 and adds the standard deviation of the portfolio to portfolio_stds
# Parameters: A list of 12 tickers
def buildportfolio(lst):
    portfolio = pd.DataFrame()
    
    #Determines how much of the portfolio the stock will be worth, based on its position in the list
    for x in range(len(lst)):
        if x <= 1:
            investment = 125000
        elif x == 2:
            investment = 62500
        else:
            investment = 62500/3
        
        #Gets closing price value from the dictionary defined earlier
        portfolio[lst[x]] = stock_price_dict[lst[x]]
        
        #Defines the number of shares based on the stock price from the first day
        print(portfolio)
        try:
            num_shares = investment / portfolio.loc[first_trading_day, lst[x]]
        except:
            num_shares = investment / portfolio[lst[x]].iloc[0]
        
        #Multiplies closing value by number of shares 
        portfolio[lst[x]] = portfolio[lst[x]] * num_shares
        
    #Drops NaN values
    portfolio = portfolio.dropna()
    
    #Adds all stocks into a final column to track the portfolio value over time
    portfolio["portfolio value"] = portfolio.sum(axis = 1)
    
    #Adds the standard deviation of the portfolio to the portfolio_stds
    portfolio_stds.append(portfolio['portfolio value'].std())
    
#Applys buildportfolio to every list of tickers in portfolios
for n in portfolios:
    buildportfolio(n)

                  LNC
Date                 
2022-11-02  52.099998
2022-11-03  34.830002
2022-11-04  34.419998
2022-11-07  33.259998
2022-11-08  34.520000
2022-11-09  32.810001
2022-11-10  34.630001
2022-11-11  35.240002
2022-11-14  33.230000
2022-11-15  34.160000
2022-11-16  35.389999
2022-11-17  36.150002
2022-11-18  37.730000
2022-11-21  38.160000
2022-11-22  38.889999
2022-11-23  38.740002
2022-11-25  38.849998
2022-11-28  37.560001
2022-11-29  38.090000
2022-11-30  38.939999
2022-12-01  38.730000
2022-12-02  38.820000
2022-12-05  37.740002
2022-12-06  37.349998
2022-12-07  35.279999
2022-12-08  31.450001
2022-12-09  31.660000
2022-12-12  32.279999
2022-12-13  32.480000
2022-12-14  32.380001
2022-12-15  30.820000
2022-12-16  29.170000
2022-12-19  29.240000
2022-12-20  29.240000
2022-12-21  30.290001
2022-12-22  29.580000
2022-12-23  30.129999
2022-12-27  29.790001
2022-12-28  29.530001
2022-12-29  30.650000
2022-12-30  30.719999
2023-01-03  30.820000
2023-01-04  31.469999
2023-01-05

2023-01-06  151687.198921  140507.974879  316.670013
                     MRNA           NVDA          EPAM        SIVB
Date                                                              
2022-11-02  125000.000000  125000.000000  62500.000000  213.050003
2022-11-03  124058.004345  126910.133364  57546.570530  207.619995
2022-11-04  133234.094391  133860.347742  59747.028359  213.199997
2022-11-07  137969.316609  135231.477292  60414.244461  212.910004
2022-11-08  138448.733362  138077.768341  61123.514179  218.419998
2022-11-09  137271.238794  130267.032267  58747.169782  208.300003
2022-11-10  141770.962098  148933.350756  66998.424273  230.949997
2022-11-11  143991.389595  154389.516281  68868.147443  235.619995
2022-11-14  150576.978756  154086.914610  68460.936811  219.759995
2022-11-15  155993.479439  157595.128777  67319.602353  239.929993
2022-11-16  155808.442745  150446.330249  66828.272819  235.050003
2022-11-17  154672.991470  148243.060535  64847.668825  222.619995
2022-11-1

2023-01-06  20847.511522  22727.271988  19855.816399  21651.278415  60.959999  
                     SEDG           ENPH          TROW          NVDA  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  129234.071982  127854.052397  60445.727641  21151.688894   
2022-11-04  125815.790834  121951.641814  61407.174561  22310.057957   
2022-11-07  121392.132361  116075.123736  62565.684704  22538.579549   
2022-11-08  144619.216845  120928.327860  65145.466583  23012.961390   
2022-11-09  143177.221668  122949.050086  63969.042016  21711.172044   
2022-11-10  162968.799430  133527.633851  74437.462476  24822.225126   
2022-11-11  165979.180779  126083.769329  79626.882090  25731.586047   
2022-11-14  163164.127778  128644.215216  74945.057573  25681.152435   
2022-11-15  165439.148128  129848.882118  75452.652669  26265.854796   
2022-11-16  166467.510414  131770.290447  73971.669222  

2023-01-06  22352.139664  22056.003676  19855.816399  20847.511522  42.349998  
                      WBD           PARA          DISH           VFC  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  118000.788436  120681.940147  58934.708278  20545.455181   
2022-11-04  102819.401165  115917.204034  57345.361712  21401.515152   
2022-11-07  101242.115641  121873.138376  63015.463419  21371.211428   
2022-11-08   99171.926040  123436.562088  65893.470582  21628.787301   
2022-11-09   97003.153744  115619.408027  60781.784506  20378.787590   
2022-11-10  106466.876289  123362.111311  68986.251097  22674.242655   
2022-11-11  116719.241597  140038.701908  73024.054026  25257.575873   
2022-11-14  113268.922462  137656.333851  70833.331422  25083.333796   
2022-11-15  111691.636938  144729.001445  68900.346672  25340.909669   
2022-11-16  107551.257737  135050.613464  67525.773458  

2023-01-06  22352.139664  148.589996  
                     MTCH           EPAM          SIVB           CZR  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  116938.126764  115093.141060  60907.061765  20332.349644   
2022-11-04  117566.678179  119494.056718  62544.001964  20152.751445   
2022-11-07  121693.258486  120828.488922  62458.930008  20828.606069   
2022-11-08  120518.137450  122247.028359  64075.332974  21102.729730   
2022-11-09  117348.046160  117494.339565  61106.547779  20303.991464   
2022-11-10  133471.792633  133996.848546  67751.112896  23924.302797   
2022-11-11  140577.166427  137736.294885  69121.095911  24633.241061   
2022-11-14  133034.539022  136921.873622  64468.432105  24344.939212   
2022-11-15  141888.927262  134639.204706  70385.469737  25663.564817   
2022-11-16  134838.211472  133656.545638  68953.883972  23976.291891   
2022-11-17  128525.352682

...

2023-01-06  20500.402991  22056.003676  15.640000  
                      CZR           TROW            ON           AMD  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  121994.097864  120891.455283  61543.418343  21359.230028   
2022-11-04  120916.508673  122814.349122  65702.460058  22098.327610   
2022-11-07  124971.636412  125131.369407  68073.116532  22414.577535   
2022-11-08  126616.378378  130290.933165  69757.523350  22688.184842   
2022-11-09  121823.948786  127938.084032  66367.910064  21291.715375   
2022-11-10  143545.816780  148874.924953  75684.162552  24329.836820   
2022-11-11  147799.446367  159253.764180  78034.023342  25715.646650   
2022-11-14  146069.635271  149890.115145  75725.753922  26127.834669   
2022-11-15  153981.388899  150905.305338  77254.203021  27136.989509   
2022-11-16  143857.751347  147943.338444  73084.761481  25832.905375   
2022-11-17  

2023-01-06  22056.003676  22727.271988  20847.511522  21651.278415  245.789993  
                      VFC           PARA          TROW          DISH  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  123272.731087  120681.940147  60445.727641  19644.902759   
2022-11-04  128409.090909  115917.204034  61407.174561  19115.120571   
2022-11-07  128227.268566  121873.138376  62565.684704  21005.154473   
2022-11-08  129772.723805  123436.562088  65145.466583  21964.490194   
2022-11-09  122272.725539  115619.408027  63969.042016  20260.594835   
2022-11-10  136045.455933  123362.111311  74437.462476  22995.417032   
2022-11-11  151545.455239  140038.701908  79626.882090  24341.351342   
2022-11-14  150500.002774  137656.333851  74945.057573  23611.110474   
2022-11-15  152045.458013  144729.001445  75452.652669  22966.782224   
2022-11-16  146227.264404  135050.613464  73971.669222 

2023-01-06  20847.511522  20180.140938  42.349998  
                       ON            AMD          NVDA          TROW  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  123086.836686  128155.380171  63455.066682  20148.575880   
2022-11-04  131404.920115  132589.965661  66930.173871  20469.058187   
2022-11-07  136146.233064  134487.465210  67615.738646  20855.228235   
2022-11-08  139515.046699  136129.109054  69038.884170  21715.155528   
2022-11-09  132735.820128  127750.292251  65133.516133  21323.014005   
2022-11-10  151368.325105  145979.020922  74466.675378  24812.487492   
2022-11-11  156068.046685  154293.879900  77194.758140  26542.294030   
2022-11-14  151451.507843  156767.008016  77043.457305  24981.685858   
2022-11-15  154508.406042  162821.937053  78797.564388  25150.884223   
2022-11-16  146169.522962  154997.432253  75223.165125  24657.223074   
2022-11-17  

2023-01-06  20847.511522  23417.995813  100.750000  
                     EPAM           TROW          MTCH            ON  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  115093.141060  120891.455283  58469.063382  20514.472781   
2022-11-04  119494.056718  122814.349122  58783.339089  21900.820019   
2022-11-07  120828.488922  125131.369407  60846.629243  22691.038844   
2022-11-08  122247.028359  130290.933165  60259.068725  23252.507783   
2022-11-09  117494.339565  127938.084032  58674.023080  22122.636688   
2022-11-10  133996.848546  148874.924953  66735.896317  25228.054184   
2022-11-11  137736.294885  159253.764180  70288.583214  26011.341114   
2022-11-14  136921.873622  149890.115145  66517.269511  25241.917974   
2022-11-15  134639.204706  150905.305338  70944.463631  25751.401007   
2022-11-16  133656.545638  147943.338444  67419.105736  24361.587160   
2022-11-17 

2023-01-06  22352.139664  19855.816399  10956.445542  62.470001  
                  TROW
Date                  
2022-11-02  104.660004
2022-11-03  101.220001
2022-11-04  102.830002
2022-11-07  104.769997
2022-11-08  109.089996
2022-11-09  107.120003
2022-11-10  124.650002
2022-11-11  133.339996
2022-11-14  125.500000
2022-11-15  126.349998
2022-11-16  123.870003
2022-11-17  122.769997
2022-11-18  122.070000
2022-11-21  121.000000
2022-11-22  124.430000
2022-11-23  124.400002
2022-11-25  125.050003
2022-11-28  121.129997
2022-11-29  120.500000
2022-11-30  124.910004
2022-12-01  127.089996
2022-12-02  126.010002
2022-12-05  123.629997
2022-12-06  121.330002
2022-12-07  121.250000
2022-12-08  122.260002
2022-12-09  119.830002
2022-12-12  120.989998
2022-12-13  123.470001
2022-12-14  118.580002
2022-12-15  111.959999
2022-12-16  110.519997
2022-12-19  109.529999
2022-12-20  109.309998
2022-12-21  113.050003
2022-12-22  110.279999
2022-12-23  111.120003
2022-12-27  110.730003
2022-12-28  10

In [17]:
#Gets the list of tickers from portfolios based on the position of the largest standard deviation in portfolio_stds, since order is preserved
finalport_tickerslst = portfolios[portfolio_stds.index(max(portfolio_stds))]

finalport_pricelst = []

# Defines finalport_pricelst as the price of each ticker at the required date
for i in range(len(finalport_tickerslst)):
    finalport_pricelst.append(stock_price_dict[finalport_tickerslst[i]].loc["2022-11-25", "Close"])
    
finalport_shareslst = []

#Defines finalport_shareslst as the number of shares of each stock, based on finalport_pricelst and the position of the ticker in the finalport_tickerslst
for n in range(len(finalport_tickerslst)):
    value = -1
    if n <= 1:
        value = 125000
    elif n == 2:
        value = 62500
    else:
        value = 62500/3
    finalport_shareslst.append(value/finalport_pricelst[n])
    
#Creates a dictionary with the data above
finalport = {"Ticker" : finalport_tickerslst,
             "Price" : finalport_pricelst,
             "Shares" : finalport_shareslst}

#Creates the index for the portfolio
index_list = list(range(1, 13))

#Creates the dataframe from the data in finalport
Portfolio_Final = pd.DataFrame(finalport, index=index_list)

#Adds a column for Value using price * shares
Portfolio_Final["Value"] = Portfolio_Final.Price * Portfolio_Final.Shares

#Adds a column for Weight based oln the value of each stock
Portfolio_Final["Weight"] = Portfolio_Final.Value / 500000 * 100

Portfolio_Final

,Ticker,Price,Shares,Value,Weight
1,SEDG,301.910004,414.030666,125000.000000,25.000000
2,ENPH,319.420013,391.334277,125000.000000,25.000000
3,TROW,125.050003,499.800068,62500.000000,12.500000
4,NVDA,162.699997,128.047534,20833.333333,4.166667
5,CZR,49.680000,419.350507,20833.333333,4.166667
6,AMD,75.139999,277.260228,20833.333333,4.166667
7,GNRC,105.250000,197.941409,20833.333333,4.166667
8,ON,73.400002,283.832873,20833.333333,4.166667
9,CDAY,67.099998,310.481875,20833.333333,4.166667
10,ETSY,120.309998,173.163775,20833.333333,4.166667


In [18]:
print("Total Value:", Portfolio_Final.Value.sum())
print("Total Weight:", Portfolio_Final.Weight.sum())

Total Value: 499999.9999999999
Total Weight: 100.0


In [19]:
#Creates another dataframe using only the Ticker and Shares column from Portfolio_Final
Stocks_Final = Portfolio_Final[["Ticker", "Shares"]]
print(Stocks_Final)
#Exports the dataframe to a csv file
Stocks_Final.to_csv('Stocks_Group_11.csv')



   Ticker       Shares
1    SEDG   414.030666
2    ENPH   391.334277
3    TROW   499.800068
4    NVDA   128.047534
5     CZR   419.350507
6     AMD   277.260228
7    GNRC   197.941409
8      ON   283.832873
9    CDAY   310.481875
10   ETSY   173.163775
11   EPAM    59.668720
12    CCL  2141.144328


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Bob Li  
Lucas Zaharia  
Charles Chen